# Plotting example for Boston 311 data

In this notebook we plot Boston's 311 data onto a map of Boston.

Data via https://data.boston.gov/dataset/311-service-requests

## Setup

In [ ]:
lapply(c('hexbin', 'skimr'),
       function(pkg) { if(! pkg %in% installed.packages()) { install.packages(pkg)} } )

In [ ]:
library(httr)
library(skimr)
library(tidyverse)

In [ ]:
options(repr.plot.height = 14, repr.plot.width = 14)
theme_set(theme_gray(base_size = 18))

In [ ]:
NUMBER_OF_LOCALITY_BINS = 90

# Download data

Data comes via a link from https://data.boston.gov/dataset/311-service-requests
It yields a 403 error but that page with the 403 also provides you with a resolved URL that 
contains a session token to download the raw data. By parsing the header, you can extract that 
resolved URL.

In [ ]:
lookup <- HEAD("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/311_service_requests_2018.csv")
resolved_url <- ((lookup$all_headers[[1]])$headers)$location

In [ ]:
dat <- read_csv(resolved_url)

In [ ]:
print(skim(dat))

In [ ]:
head(dat)

# Examine count of reports per locality

In [ ]:
ggplot(dat) +
  geom_hex(aes(longitude, latitude, col=..count..), bins = NUMBER_OF_LOCALITY_BINS)

In [ ]:
ggplot(dat[dat$longitude < -71.058 & dat$longitude > -71.059 & dat$latitude < 42.36 & dat$latitude > 42.359,]) +
  geom_hex(aes(longitude, latitude, col=..count..), bins = NUMBER_OF_LOCALITY_BINS)

In [ ]:
length(dat[dat$latitude==42.3594 & dat$longitude==-71.0587,1])

It looks like any report without GPS coordinates gets assigned to City Hall.

Let's remove City Hall and see how the report density actually varies:

In [ ]:
notCityHall <- dat[!(dat$latitude==42.3594 & dat$longitude==-71.0587),]

In [ ]:
ggplot(notCityHall) +
  geom_hex(aes(longitude, latitude, col=..count..), bins = NUMBER_OF_LOCALITY_BINS)

# Needle pickup requests

In [ ]:
needleCases <- notCityHall[grep("Needle", notCityHall$case_title), ]

In [ ]:
nrow(needleCases)

In [ ]:
ggplot(needleCases) +
  geom_hex(aes(longitude, latitude, col=..count..), bins = NUMBER_OF_LOCALITY_BINS) + 
  scale_colour_gradient(low = "blue", high = "orange") +
  scale_fill_gradient(low = "blue", high = "orange") +
  ggtitle("BOS311 Needle reports")

In [ ]:
ggplot(needleCases[with(needleCases, latitude>42.33 & latitude < 42.34 & longitude > -71.085 & longitude < -71.06),]) +
  geom_hex(aes(longitude, latitude, col=..count..), bins=60) + 
  scale_colour_gradient(low = "blue", high = "orange") +
  scale_fill_gradient(low = "blue", high = "orange") +
  ggtitle("BOS311 Needle report hotspot")

In [ ]:
# Note: this is a failed overlay of the generic request density map and the needle pickup map
ggplot(notCityHall) + 
  geom_hex(aes(longitude, latitude, alpha=0.5, col=..count..), bins = NUMBER_OF_LOCALITY_BINS) +
  geom_hex(data=needleCases, aes(longitude, latitude, alpha=0.5, col=..count..), bins = NUMBER_OF_LOCALITY_BINS) +
  scale_colour_gradient(low = "blue", high = "orange") +
  scale_fill_gradient(low = "blue", high = "orange")

# What else can we plot?

In [ ]:
top_case_types <- notCityHall %>%
    group_by(type) %>%
    summarize(
        count = n()
    ) %>%
    arrange(desc(count)) %>%
    head(n=20)

top_case_types

In [ ]:
plotBOS <- function(df, case_type) {
  p <- ggplot(notCityHall %>% filter(type == case_type)) +
    geom_hex(aes(longitude, latitude, col=..count..), bins = NUMBER_OF_LOCALITY_BINS) + 
    scale_colour_gradient(low = "blue", high = "orange") +
    scale_fill_gradient(low = "blue", high = "orange") +
    ggtitle(case_type)
  print(p)
}

In [ ]:
for (x in top_case_types$type) {
    plotBOS(notCityHall, case_type = x)
}

# Provenance

In [ ]:
devtools::session_info()

Copyright 2018 The Broad Institute, Inc., Verily Life Sciences, LLC All rights reserved.

This software may be modified and distributed under the terms of the BSD license. See the LICENSE file for details.